In [40]:
import tensorflow as tf
import numpy as np

## Problem 2 : 분류기 만들기
 - 전처리한 데이터 불러오기. training과 test set으로 구분되어 있음
   - Training data set:28000개, Test data set: 12000개
 - 전처리한 데이터는 연속형, 범주형 데이터로만 이루어진다. 범주형 데이터는 더미변수를 적용하여 변환되어 있음
   - 데이터 범위 : 0~1로 표준화
   - x2, x3을 더미형으로 추가했기 때문에 변수는 15개에서 28개로 늘어남

In [41]:
xy_train = np.loadtxt("data_for_training_v1.csv", delimiter=",", dtype=np.float32)
xy_test = np.loadtxt("data_for_test_v1.csv", delimiter=",", dtype=np.float32)
x_train = xy_train[:,0:-1]
y_train = xy_train[:,[-1]]
x_test = xy_test[:,0:-1]
y_test = xy_test[:,[-1]]

## Logistic Classification 
- Logistic Classification 선택 이유
  - Y값이 continuous한 경우 linear regression, 여러개 level한 경우는 Softmax 등을 사용할 수 있음
  - 허나 Y가 0/1인 경우 상기 방법을 사용할 경우 outlier에 따라 오동작할 확률이 높음
  - Y값이 0과 1로 표현할 수 있는 on/off 변수이기 때문에 logistic classification을 사용

- Logistic Regression의 cost function을 수식으로 표현하고 (hypythesis) 이를 최소화 시키는 cost function을 만듦 (cost)

- Learning Rate를 적당히 조절함
  : input data의 스케일이 다를 경우 결과값이 발산할 수 있음
  : input data는 0~1 사이의 표준화된 데이터로 발산할 확률은 적지만, 목적함수의 최소값을 찾을 수 있도록 loop를 많이 돌려야 함

In [42]:
#placeholders for a tensor that will be always fed
X = tf.placeholder(tf.float32, shape=[None, 28]) # input 1~28
Y = tf.placeholder(tf.float32, shape=[None, 1]) # input 1

W = tf.Variable(tf.random_normal([28,1]), name='weight') #variable 28개
b = tf.Variable(tf.random_normal([1]), name='bias') #bias 1개

# Hypothesis using sigmoid:
hypothesis = tf.sigmoid(tf.matmul(X,W) + b)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y)*tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

## Training & Test
 - train을 feed하고 Test case에 적용해서 accuracy를 뽑아냄, train의 결과가 수렴하는지 확인하기 위해 print를 넣음. 
 - 실험결과 약 30000번의 iteration 후 수렴 >> 테스트마다 수치가 약간씩 변화하지만 Accuracy 약 88%
 - Iteration 횟수 30000번을 선택한 이유
   : cost의 변화량이 줄어드는 시점의 iteration 값을 추출
   : 하기 실험의 경우 약 30000번에서부터 cost의 변화량이 0에 가까워진다고 볼 수 있음
   : 하기 코드는 50000으로 하였지만 30000으로 해도 비슷한 결과가 도출
 ## 분류기 정확도 : 약 88%

In [43]:
# Launch Graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    feed_train = {X:x_train, Y:y_train} # training
    feed_test = {X:x_test, Y:y_test} # training
    for step in range(50001):
        sess.run(train, feed_dict=feed_train)
        if step % 2000 == 0:
            print("Step: ", step)
            c, a = sess.run([cost, accuracy], feed_dict=feed_train)
            print("Training: [cost = ", c, " accuracy = ", a, "]")
            c, a = sess.run([cost, accuracy], feed_dict=feed_test)
            print("Test: [cost = ", c, " accuracy = ", a, "]")
            

    # Result : TEST  case
    feed = {X: x_test, Y: y_test}
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict = feed)
    print("\nHypothesis: ", h)
    print("\nCorrect (Y): ", c)
    print("\nTEST Accuracy: ", a)

Step:  0
Training: [cost =  2.5414045  accuracy =  0.13432142 ]
Test: [cost =  2.5487869  accuracy =  0.13083333 ]
Step:  2000
Training: [cost =  0.6305362  accuracy =  0.6694643 ]
Test: [cost =  0.625946  accuracy =  0.672 ]
Step:  4000
Training: [cost =  0.46909922  accuracy =  0.8356786 ]
Test: [cost =  0.46081176  accuracy =  0.8415 ]
Step:  6000
Training: [cost =  0.44264165  accuracy =  0.8626071 ]
Test: [cost =  0.43329814  accuracy =  0.8666667 ]
Step:  8000
Training: [cost =  0.4323194  accuracy =  0.868 ]
Test: [cost =  0.4227185  accuracy =  0.87233335 ]
Step:  10000
Training: [cost =  0.42528397  accuracy =  0.87110716 ]
Test: [cost =  0.4157366  accuracy =  0.87516665 ]
Step:  12000
Training: [cost =  0.41932583  accuracy =  0.8723214 ]
Test: [cost =  0.4099611  accuracy =  0.87691665 ]
Step:  14000
Training: [cost =  0.41396046  accuracy =  0.8732143 ]
Test: [cost =  0.4048286  accuracy =  0.87766665 ]
Step:  16000
Training: [cost =  0.40905094  accuracy =  0.8740357 ]
Te

## 결과 (전처리 > 표준화 > Logistic Classification >>> Accuracy 88%)
 - 데이터 전처리
   - 범주형 데이터는 더미변수를 통해 차원을 늘림
   - 변수 표준화 (0 ~ 1. min max scaler 사용)
 - Logistic Classification
   - Iteration 횟수
     - cost가 수렴하는 시점의 Iteration을 선택
   - learning Rate 
     - training 후 업데이트 될 때 변화 시키는 단위
     - 순차적 탐색으로 찾아낼 수 있음
     - 모든 변수의 범위를 표준화 시켰기 때문에 NaN이 출력될 확률은 높지 않음
 - 분류기 정확도: 약 88%